In [138]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/NLP

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/NLP


In [139]:
%pip install transformers

In [140]:
!nvidia-smi

Wed Nov 17 05:20:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    60W / 149W |   4882MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [141]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt

from BERT import TweetDataset, train, generate, save_model, load_model, collate_batch
from transformers import BertLMHeadModel,  BertTokenizer, BertConfig, BertLMHeadModel
from transformers import AdamW, __version__ 

from tqdm import tqdm

import nltk
nltk.download('punkt')
torch.cuda.empty_cache()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [142]:
device = torch.device('cuda')
BOS_TOKEN = "<BOS>"
EOS_TOKEN = "<EOS>"
PAD_TOKEN = "<PAD>"
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', bos_token=BOS_TOKEN, eos_token=EOS_TOKEN, pad_token=PAD_TOKEN)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [143]:
epochs = 5
learning_rate = 5e-4
epsilon = 1e-8
sample_every = 100
batch_size = 8
configuration = BertConfig.from_pretrained("bert-base-cased")
configuration.is_decoder=True

# instantiate the model
model = BertLMHeadModel.from_pretrained("bert-base-cased", config=configuration).to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=epsilon)

# this step is necessary because I've added some tokens to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))
model.cuda()

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [144]:
def train_loss_plot(training_stats):
    # Display floats with two decimal places.
    pd.set_option('precision', 2)
    df_stats = pd.DataFrame(data=training_stats)
    df_stats = df_stats.set_index('epoch')

    # Plot the loss
    sns.set(style='darkgrid')
    sns.set(font_scale=1.5)
    plt.rcParams["figure.figsize"] = (12,6)
    plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
    plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")
    plt.title("Training & Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.xticks([1, 2, 3, 4])
    plt.show()

In [145]:
train_dataset = TweetDataset(tokenizer, 'data/elonmusk/train.txt')
val_dataset = TweetDataset(tokenizer, 'data/elonmusk/test.txt')

11/17/2021 05:20:53 - INFO - Creating features from dataset file at data/elonmusk/train.txt
11/17/2021 05:20:55 - INFO - Creating features from dataset file at data/elonmusk/test.txt


In [146]:
train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset), # Select batches randomly
    batch_size=batch_size,
)

val_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset), # Pull out batches sequentially.
    batch_size=batch_size,
)

In [147]:
training_stats = train(model, device, optimizer, train_dataloader, val_dataloader, tokenizer,
                       epochs=epochs, sample_every=sample_every)

11/17/2021 05:20:55 - INFO - ======== Epoch 1 / 5 ========
11/17/2021 05:20:55 - INFO - Training...


RuntimeError: ignored

In [ ]:
train_loss_plot(training_stats)

In [ ]:
save_model('elonmusk', model, tokenizer)

In [ ]:
model, tokenizer = load_model('elonmusk')
model = model.to(device)

In [ ]:
tweets = generate(model, device, tokenizer, n_sequences=10, max_length=50)
tweets